In [1]:
import networkx as nx
import numpy as np
import scipy.sparse as sparse
import math
import matplotlib.pyplot as plt

In [2]:
n = 20000
deg_v = 5 # w_c. Every bit is in this many checks
deg_c = 6 # w_r. Every check has this many bits in it
num_checks = (n*deg_v)//deg_c
k = n - num_checks

vs = np.array([[j for i in range(deg_v)] for j in range(n)]).flatten()
cs = np.array([[j for i in range(deg_c)] for j in range(num_checks)]).flatten()

H = np.zeros((num_checks, n), dtype=bool)

while (vs.size and cs.size):
    # choose random 'stub' from each array
    double_edge = True
    while(double_edge):
        v_ind = np.random.randint(0, len(vs))
        c_ind = np.random.randint(0, len(cs))

        if (H[cs[c_ind]][vs[v_ind]] != 1):
            double_edge = False
            H[cs[c_ind]][vs[v_ind]] = 1
            # if (girth(H, 4)):
            #     H[cs[c_ind]][vs[v_ind]] = 0
            #     break
            vs = np.delete(vs, v_ind)
            cs =np.delete(cs, c_ind)
            
H = sparse.csc_matrix(H)
print(num_checks)

16666


In [3]:
G = nx.algorithms.bipartite.matrix.from_biadjacency_matrix(H)

In [4]:
bits = nx.bipartite.sets(G)[1]
checks = nx.bipartite.sets(G)[0]
# pos = nx.bipartite_layout(G, bits)
# nx.draw(G, pos)

In [5]:
p = 0.30
for check in checks:
    if (np.random.rand() < p):
        G.remove_node(check)

In [6]:
degs = [0 for _ in range(deg_v+1)]
deg_seq = G.degree()
for bit in bits:
    degs[deg_seq[bit]] += 1
degs

[50, 600, 2692, 6180, 7260, 3218]

In [7]:
def hgd(N,K,n,k):
    return (math.comb(K,k) * math.comb(N-K,n-k)) / math.comb(N,n)

probs = [hgd(num_checks, deg_v, int(p*num_checks), deg_v - i) for i in range(deg_v+1)]
print(probs)
print([deg/n for deg in degs])

[0.0024246574065622125, 0.02831679475711845, 0.13224328568316407, 0.30870881078529294, 0.360222045718253, 0.1680844056496093]
[0.0025, 0.03, 0.1346, 0.309, 0.363, 0.1609]


In [202]:
nc = np.arange(1e4, 1e5, 1e4, dtype=np.int64)

probs = [hgd(c, deg_v, int(0.1*c), deg_v-0) for c in nc]

In [6]:
import numpy as np

L = 4
eigs = np.random.uniform(0, 100, [L])
# eigs = [max(eigs) - e for e in eigs]
# eigs = [-0.   , 11.432, 33.894, 38.803, 53.467]
print(eigs)

[28.36074745 33.55065526 77.44030686 45.87024657]


In [7]:
diffs = []
for i in eigs:
    for j in eigs:
        diffs.append(i - j)
diffs = np.array(list(set(diffs)))
diffs = np.sort(diffs)
# diffs = [d for d in diffs if d >= 0]
diffs = diffs[np.searchsorted(diffs, 0)+1 :]
diffs

array([ 5.1899078 , 12.31959131, 17.50949911, 31.5700603 , 43.88965161,
       49.07955941])

In [10]:
lambda_n = diffs[-1]
frequencies1 = set([0, lambda_n])
frequencies2 = set([0, lambda_n])

diffs2 = lambda_n - diffs
pairings = []
temp_pairings = [(i,j) for i,j in zip(diffs, diffs2)]

for i, pair1 in enumerate(temp_pairings): # can be improved likely
    for j, pair2 in enumerate(temp_pairings):
        if (i != j):
            if (np.any(np.isclose(pair1, pair2[::-1]))):
                pairings.append(pair1)

lambda_1, lambda_2 = pairings[0]
frequencies1.add(lambda_1)
frequencies2.add(lambda_2)

for pair in pairings:
    if (np.any(np.isclose(abs(lambda_1 - pair[0]), diffs))):
        frequencies1.add(pair[0])
    if (np.any(np.isclose(abs(lambda_2 - pair[1]), diffs))):
        frequencies2.add(pair[1])

frequencies1 = np.sort(list(frequencies1))
frequencies2 = np.sort(list(frequencies2))
print(frequencies1)
print(frequencies2)

[ 0.          5.1899078  17.50949911 49.07955941]
[ 0.         31.5700603  43.88965161 49.07955941]
